### Canny edge detector:

The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a wide range of edges in images.
Canny edge detection is a technique to extract useful structural information from different vision objects and dramatically reduce the amount of data to be processed.

This removes most of the background noise from the image and turns the text regions into bright clumps of edges. (edges in white and other stuff in black)

#### Parameters: (TWO)
    1. size of the Gaussian filter (smoothing filter) - 
            used in the first stage directly affects the results of the Canny algorithm. 
                Smaller filters cause less blurring, and allow detection of small, sharp lines. 
                A larger filter causes more blurring.
    2. Thresholds: 
            use of two thresholds with hysteresis allows more flexibility than in a single-threshold approach.
            But general problems of thresholding approaches still apply. 
                A threshold set too high can miss important information. 
                A threshold set too low will falsely identify irrelevant information (such as noise) as important.
            

### Rank Filter:

This essentially replaces a pixel with something like the median of the pixels to its left and right. 

#### Rank filters can be used for several purposes such as:
    1. image quality enhancement e.g. image smoothing, sharpening
    2. image pre-processing e.g. noise reduction, contrast enhancement
    3. feature extraction e.g. border detection, isolated point detection
    4. post-processing e.g. small object removal, object grouping, contour smoothing

    
![6_rank_filter.jpg](img/6_rank_filter.jpg)

##### How it detects border
The text areas have lots of white pixels, but the borders consist of just a thin, 1 pixel line. The areas around the borders will be mostly black, so the rank filter will eliminate them.

##### Limitations:
While this is effective, it still leaves bits of text outside the borders (look at the top left and bottom right). 


##### Solution - Contours

### Contours: closed curves

Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition.


#### Pre-requisites:
    1. For better accuracy, use binary images. So before finding contours, apply threshold or canny edge detection
    2. In OpenCV, finding contours is like finding white object from black background. 


These are sets of white pixels which are connected to one another. The border contours are easy to pick out: they’re the ones whose bounding box covers a large fraction of the image.

    th,contours, h = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) # h would have heirarchy of contours

Above API is very sensitive to noise, so in order to remove all the noise we would blur the image before passing it to contours.

        cv2.pyMeanShiftFiltering(image, 21, 51)
        
How to draw contours:
        cv2.drawContours(image, contours_ds, MINUS_ONE_signifies_to_draw_all_contours, (0,0,255), width_integer)
        
        
#### Third parameter in findContours method - Contour Approximation Method
     Contours are the boundaries of a shape with same intensity. 
     It stores the (x,y) coordinates of the boundary of a shape. But does it store all the coordinates ? 
     That is specified by this contour approximation method.
     
     If you pass cv2.CHAIN_APPROX_NONE, all the boundary points are stored.
     
     But actually do we need all the points? For eg, you found the contour of a straight line. 
     Do you need all the points on the line to represent that line? No, we need just two end points of that line. 
     This is what cv2.CHAIN_APPROX_SIMPLE does. 
     It removes all redundant points and compresses the contour, thereby saving memory.
     
     
     
     
![7_region_of_interest_using_contours.jpg](img/7_region_of_interest_using_contours.jpg)

With polygons for the borders, it’s easy to black out everything outside them.

What we’re left with is an image with the text and possibly some other bits due to smudges or marks on the original page.

At this point, we’re looking for a crop (x1, y1, x2, y2) which:
    1. maximizes the number of white pixels inside it and
    2. is as small as possible.
    
These two goals are in opposition to one another. If we took the entire image, we’d cover all the white pixels. But we’d completely fail on goal #2: the crop would be unnecessarily large. This should sound familiar: it’s a classic precision/recall tradeoff:

    1. The recall is the fraction of white pixels inside the cropping rectangle.
    2. The precision is the fraction of the image outside the cropping rectangle.

A fairly standard way to solve precision/recall problems is to optimize the F1 score, the harmonic mean of precision and recall. This is what we’ll try to do.

# OR, we can simplify the solution

We can simplify the problem by finding individual chunks of text. To do this, we apply binary dilation to the de-bordered edge image. This “bleeds” the white pixels into one another. We do this repeatedly until there are only a few connected components.



Dilation vs Erosion

    1, Dialiation:
        1. The basic effect of the operator on a binary image is to gradually enlarge the boundaries of regions of foreground pixels (i.e. white pixels, typically). Thus areas of foreground pixels grow in size while holes within those regions become smaller.